In [134]:
import pandas as pd
import re
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel, TfidfModel, Word2Vec, LsiModel, LdaMulticore, HdpModel
import numpy as np
import tqdm
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from pprint import pprint

In [135]:
raw_sop = pd.read_csv('./data/sops.csv')

In [136]:
raw_sop.head()

,city,sop_doc
0,Austin,Austin Police DepartmentPolicy ManualCopyright...
1,Austin,Austin Police DepartmentPolicy ManualCopyright...
2,Austin,Austin Police DepartmentPolicy ManualCopyright...
3,Austin,Austin Police DepartmentPolicy ManualCanons of...
4,Austin,Austin Police DepartmentPolicy ManualCanons of...


In [137]:
sop = raw_sop.drop(columns=['city'])

In [138]:
sop.sample(10)

,sop_doc
171,Austin Police DepartmentPolicy ManualDetention...
5203,Line/Operations Procedures\n - Inform the res...
4229,UNIFORM POLICY\n - HONOR GUARD UNIFORM \n Op...
829,Dallas Police Department General Order\n \n301...
4005,TRANSFER POLICY\n Operations Order\n 3.2\n PHO...
5390,Line/Operation\ns Procedures\n L 4416 TAGGING...
6300,Las Vegas Metropolitan Polic...
1020,Dallas Police Department General Order \n428....
784,\nDallas Police Department General Order\n \n...
5012,Administrative Structure\n A 2300 \n- WRITTEN...


In [139]:
sop['sop_doc_processed'] = sop[
    'sop_doc'
].map(lambda x: re.sub('[,\.!?]', '', str(x))).map(lambda x: re.sub('\n', ' ', str(x)))

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<ipython-input-139-da76bbe04fda>:3: DeprecationWarning: invalid escape sequence \.
  ].map(lambda x: re.sub('[,\.!?]', '', str(x))).map(lambda x: re.sub('\n', ' ', str(x)))


In [140]:
sop['sop_doc_processed'] = sop['sop_doc_processed'].map(lambda x: x.lower().strip())

In [141]:
sop = sop[sop['sop_doc_processed'] != '']

In [142]:
sop.sample(10)

,sop_doc,sop_doc_processed
2782,Evanston Police Department\nPolicy Manual\nPee...,evanston police department policy manual peer ...
5907,Las Vegas Metropolitan Police Department Partn...,las vegas metropolitan police department partn...
1724,EYE\n COLOR\n This\n field\n is used\n to desc...,eye color this field is used to describe ...
2653,Evanston Police Department\nPolicy Manual\nRec...,evanston police department policy manual recru...
2146,Evanston Police Department\nPolicy Manual\nSea...,evanston police department policy manual searc...
1791,September 2015\n 578 Impairing Substance: Alc...,september 2015 578 impairing substance: alco...
4033,TRANSFER POLICY \n- QUALIFICATION CHARTS\n O...,transfer policy - qualification charts ope...
193,Austin Police DepartmentPolicy ManualCare and ...,austin police departmentpolicy manualcare and ...
5192,Line/Operations Procedures\n - Competen...,line/operations procedures - competent...
4737,ARIZONA TRAFFIC TICKET AND COMPLAINT (ATTC)\n ...,arizona traffic ticket and complaint (attc) o...


In [143]:
sop.shape

(4883, 2)

In [144]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [145]:
data = sop.sop_doc_processed.values.tolist()
data_words = np.array(list(sent_to_words(data)))

In [146]:
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=1) # higher threshold fewer phrases.

In [147]:
trigram = gensim.models.Phrases(bigram[data_words], threshold=1)

In [148]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [149]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [150]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [151]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([
            token.lemma_ for token in doc if token.pos_ in allowed_postags
        ])
    return texts_out

In [152]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [153]:
print(data_lemmatized[:1])

[['right', 'permission', 'departmentchief', 'message', 'chief', 'proud', 'present', 'new', 'edition', 'manual', 'design', 'assist', 'employee', 'accomplish', 'professional', 'adherence', 'policy', 'also', 'help', 'safeguard', 'employee', 'ensure', 'employee', 'protect', 'individual', 'action', 'scrutinize', 'especially', 'critical', 'incidentall', 'employee', 'abide', 'policy', 'responsible', 'keep', 'current', 'content', 'safebrianbrian', 'manleychief', 'police']]


In [154]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Tfidf corpus
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [155]:
tfidf

In [156]:
hdp_model_bow = HdpModel(
    corpus=corpus,
    id2word=id2word
)

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [157]:
hdp_model_tfidf = HdpModel(
    corpus=corpus_tfidf,
    id2word=id2word
)

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [158]:
pprint(hdp_model_bow.print_topics())

[(0,
  '0.013*officer + 0.008*department + 0.007*employee + 0.007*report + '
  '0.006*may + 0.006*person + 0.006*member + 0.005*shall + 0.005*request + '
  '0.005*police'),
 (1,
  '0.009*employee + 0.008*officer + 0.008*department + 0.006*supervisor + '
  '0.006*police + 0.005*member + 0.005*report + 0.005*chief + 0.005*may + '
  '0.004*shall'),
 (2,
  '0.011*officer + 0.008*department + 0.007*employee + 0.005*supervisor + '
  '0.005*shall + 0.004*vehicle + 0.004*police + 0.004*report + 0.004*include + '
  '0.004*person'),
 (3,
  '0.007*officer + 0.006*employee + 0.004*department + 0.004*report + '
  '0.004*may + 0.003*require + 0.003*person + 0.003*must + 0.003*shall + '
  '0.003*member'),
 (4,
  '0.005*officer + 0.003*employee + 0.003*pursuit + 0.002*shall + 0.002*person '
  '+ 0.002*require + 0.002*supervisor + 0.002*permission + 0.002*chief + '
  '0.002*department'),
 (5,
  '0.004*officer + 0.002*employee + 0.002*report + 0.002*policy + 0.002*shall '
  '+ 0.002*department + 0.002*i

In [159]:
pprint(hdp_model_tfidf.print_topics())

[(0,
  '0.002*victim + 0.001*member + 0.001*report + 0.001*officer + 0.001*employee '
  '+ 0.001*suspect + 0.001*person + 0.001*department + 0.001*procedure + '
  '0.001*request'),
 (1,
  '0.001*vehicle + 0.001*officer + 0.001*pursuit + 0.001*information + '
  '0.001*report + 0.001*record + 0.001*service + 0.001*order + 0.001*arrest + '
  '0.001*member'),
 (2,
  '0.001*employee + 0.001*vehicle + 0.001*officer + 0.001*report + 0.001*duty '
  '+ 0.001*police + 0.001*supervisor + 0.001*member + 0.001*office + '
  '0.001*chief'),
 (3,
  '0.000*member + 0.000*officer + 0.000*request + 0.000*shall + 0.000*person + '
  '0.000*supervisor + 0.000*order + 0.000*use + 0.000*department_general + '
  '0.000*training'),
 (4,
  '0.000*employee + 0.000*order + 0.000*property + 0.000*request + '
  '0.000*search + 0.000*shall + 0.000*item + 0.000*victim + 0.000*time + '
  '0.000*person'),
 (5,
  '0.000*adult_passenger + 0.000*selection_hire + 0.000*interrupt_disrupt + '
  '0.000*fence + 0.000*either_exp

In [160]:
coherence_model_lda_bow = CoherenceModel(model=hdp_model_bow, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [161]:
coherence_model_lda_bow.get_coherence()

0.47311390134478454

In [162]:
coherence_model_lda_tfidf = CoherenceModel(model=hdp_model_tfidf, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [163]:
coherence_model_lda_tfidf.get_coherence()

0.6981581633742951

In [164]:
hdp_model_tfidf.get_topics().shape

(150, 42785)

In [165]:
lda_model = LdaMulticore(corpus_tfidf, num_topics=30, id2word=id2word, workers=3)

In [166]:
pprint(lda_model.print_topics())

[(24,
  '0.001*"report" + 0.001*"officer" + 0.001*"vehicle" + 0.001*"employee" + '
  '0.001*"request" + 0.001*"search" + 0.001*"member" + 0.001*"person" + '
  '0.001*"warrant" + 0.001*"form"'),
 (26,
  '0.002*"employee" + 0.002*"vehicle" + 0.001*"officer" + 0.001*"report" + '
  '0.001*"investigation" + 0.001*"supervisor" + 0.001*"item" + 0.001*"person" '
  '+ 0.001*"request" + 0.001*"individual"'),
 (6,
  '0.001*"officer" + 0.001*"chief" + 0.001*"employee" + 0.001*"police" + '
  '0.001*"report" + 0.001*"arrest" + 0.001*"vehicle" + 0.001*"information" + '
  '0.001*"juvenile" + 0.001*"supervisor"'),
 (16,
  '0.001*"officer" + 0.001*"employee" + 0.001*"report" + 0.001*"vehicle" + '
  '0.001*"member" + 0.001*"victim" + 0.001*"person" + 0.001*"duty" + '
  '0.001*"case" + 0.001*"evidence"'),
 (20,
  '0.001*"officer" + 0.001*"arrest" + 0.001*"shall" + 0.001*"procedure" + '
  '0.001*"employee" + 0.001*"person" + 0.001*"report" + 0.001*"unit" + '
  '0.001*"prisoner" + 0.001*"duty"'),
 (11,
  '0

In [167]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [168]:
coherence_model_lda.get_coherence()

0.2735028594433341

In [169]:
lda_hdp_sug = hdp_model_tfidf.suggested_lda_model()

In [170]:
lda_hdp_sug.get_topics().shape

(150, 42785)

In [171]:
coherence_model_lda_hdp_sug = CoherenceModel(
    model=lda_hdp_sug, 
    texts=data_lemmatized, 
    dictionary=id2word, 
    coherence='c_v'
)

In [172]:
coherence_model_lda_hdp_sug.get_coherence()

0.7192671855953345

In [173]:
hdp_model_tfidf = gensim.models.HdpModel(
    corpus=corpus_tfidf,
    id2word=id2word,
    alpha=10
)

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [174]:
hdp_model_tfidf.get_topics().shape

(150, 42785)

In [175]:
pprint(hdp_model_tfidf.print_topics())

[(0,
  '0.002*employee + 0.002*officer + 0.002*report + 0.002*vehicle + '
  '0.002*member + 0.001*department + 0.001*person + 0.001*suspect + '
  '0.001*shall + 0.001*police'),
 (1,
  '0.000*employee + 0.000*list + 0.000*victim + 0.000*person + 0.000*use + '
  '0.000*obeying + 0.000*evidence_tagge + 0.000*surrounding + 0.000*decision + '
  '0.000*report'),
 (2,
  '0.000*victim + 0.000*employee + 0.000*order + 0.000*member + 0.000*report + '
  '0.000*committed_battery + 0.000*reason_given + 0.000*aggressor_predominant '
  '+ 0.000*supervisor + 0.000*officer'),
 (3,
  '0.000*employee + 0.000*vehicle + 0.000*supervisor + 0.000*report + '
  '0.000*coope + 0.000*process + 0.000*ellaneo + 0.000*record + 0.000*search + '
  '0.000*must'),
 (4,
  '0.001*department_general + 0.001*order + 0.001*revise + 0.000*employee + '
  '0.000*report + 0.000*david_brown + 0.000*chief + 0.000*arrest + '
  '0.000*officer + 0.000*improper_political'),
 (5,
  '0.000*employee + 0.000*chief + 0.000*exists_officer 

In [191]:
lsi_model = LsiModel(corpus_tfidf, num_topics=10, id2word=id2word)

In [192]:
coherence_model_lsi = CoherenceModel(model=lsi_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [193]:
coherence_model_lsi.get_coherence()

0.3817258341906938

In [194]:
pprint(lsi_model.print_topics())

[(0,
  '0.167*"employee" + 0.142*"officer" + 0.136*"vehicle" + 0.127*"report" + '
  '0.125*"member" + 0.116*"chief" + 0.112*"shall" + 0.107*"supervisor" + '
  '0.104*"department" + 0.102*"person"'),
 (1,
  '0.732*"department_general" + 0.383*"revise" + 0.326*"order" + '
  '0.266*"police_revise" + 0.227*"david_brown" + 0.196*"chief" + '
  '0.043*"safety_revise" + 0.039*"family_support" + 0.039*"team_revise" + '
  '0.036*"personnel_table"'),
 (2,
  '0.343*"employee" + -0.268*"vehicle" + 0.230*"chief" + -0.179*"victim" + '
  '-0.171*"arrest" + -0.165*"suspect" + -0.148*"search" + 0.123*"member" + '
  '-0.119*"child" + -0.119*"pursuit"'),
 (3,
  '0.642*"vehicle" + 0.246*"pursuit" + -0.240*"victim" + -0.160*"child" + '
  '0.127*"employee" + -0.126*"investigation" + 0.116*"tow" + 0.101*"wear" + '
  '-0.100*"arrest" + -0.085*"juvenile"'),
 (4,
  '-0.529*"police_revise" + 0.527*"revise" + -0.399*"david_brown" + '
  '-0.328*"chief" + 0.166*"employee" + -0.123*"pursuit" + -0.119*"vehicle" + '
  